---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [62]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    ##
    #    What to do=> predict whether a given blight ticket will be paid on time
    ##
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import MinMaxScaler

    df = pd.read_csv('readonly/train.csv', encoding = 'ISO-8859-1')
    addresses = pd.read_csv('readonly/addresses.csv', encoding = 'ISO-8859-1')
    latlons = pd.read_csv('readonly/latlons.csv', encoding = 'ISO-8859-1')
    eval_df = pd.read_csv('readonly/test.csv', encoding = 'ISO-8859-1')

    #
    # ticket_idからlanとlonを求める
    #
    df = df.merge(addresses, on='ticket_id', how='left')
    # 以下のprint結果からticket_idが指すaddressは郵送先住所ではなく違反のある住所とわかる
    print(df[['address', 'violation_zip_code', 'violation_street_name', 'mailing_address_str_name', 'city', 'state', 'zip_code']].head())
    df = df.merge(latlons, on='address', how='left')

    #
    #  TRAIN対象のカラムを導き出す
    #
    # test時にはまだ分からないカラムをdfから除外(但し学習のtargetは含める)
    # inspector_nameを除外（人名は年月とともに変わるし、経験等、不確定要素もある為）
    # すでにlatとlonがある為、violation_zip_codeを除外。
    # 郵送先住所のcityを除外（種類が多すぎる為）
    # hearing_dateを除外（日付を含めるには、必要な情報に整形するのに時間がかかりすぎる為）
    # violation_codeを除外（情報が整理されておらず、種類が多すぎる為）
    study_columns = ['lat', 'lon', 'agency_name', 'state', 'country', 'disposition', 'fine_amount', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'compliance']
    df = df[study_columns]

    # 各自の値を確認する
    print(df.dtypes)
    print(df['compliance'].unique())
    for col in study_columns:
        if df[col].dtypes != 'float64':
            print(df[col].unique())
            print(eval_df[col].unique())

    def preprocess(df, stage):
        if stage == 'train':
            # floatでないtarget値をNaNに変換し、除外する
            df['compliance'] = pd.to_numeric(df['compliance'], errors='coerce')
#             df = df.copy() # To deal with SettingWithCopyWarning, Make a copy of the subset and modify that copy.
            df = df.dropna(subset=['compliance'])

        # floatでないlat, lon値をNaNに変換し、除外する
        latlon = ['lat', 'lon']
        df[latlon] = df[latlon].apply(pd.to_numeric, errors='coerce')
#         df = df.copy() # To deal with SettingWithCopyWarning, Make a copy of the subset and modify that copy.
#         df = df.dropna(axis=0, subset=latlon)　＃ ここでdropしてしまうと関数の返却するrow数が一致しないことになるのでfillnaを使う
        df.fillna(0,  inplace=True)

        # Normalization
        continuous = ['lat', 'lon', 'fine_amount', 'discount_amount', 'clean_up_cost', 'judgment_amount']
        scaler = MinMaxScaler()
        df[continuous] = scaler.fit_transform(df[continuous])

        # Convert to Bins
        categorical = ['agency_name','state','country','disposition']
        for column in categorical:
            one_hot = pd.get_dummies(df[column], prefix=column[0:2])
            df = df.drop(column, axis=1)
            df = df.join(one_hot)
        return df

    train_df = preprocess(df, stage='train')

    print(train_df['compliance'].unique())

    X_train = train_df.drop('compliance', axis=1)
    y_train = train_df['compliance']

    # Create Dataset for Evaluate modeling
    eval_df = eval_df.merge(addresses, on='ticket_id', how='left')
    eval_df = eval_df.merge(latlons, on='address', how='left')
    X_eval = preprocess(eval_df[study_columns[0:-1]], stage='eval')

    # TrainデータとEvalデータのカラム数を合わせる
    X_train, X_eval = X_train.align(X_eval, join='left', axis=1)

    # Training
    model = RandomForestClassifier().fit(X_train, y_train)

    # TrainデータとEvalデータのカラム数が一致することを確認
    print(X_train.shape[1] == X_eval.shape[1])

    # TrainデータにありEvalデータに無いカラムのNaNを0にする
    X_eval.fillna(0,  inplace=True)

    predictions = model.predict_proba(X_eval)

    answer = pd.DataFrame(predictions, index=eval_df['ticket_id'], columns=[model.classes_[0], 'compliance'])
    answer = answer['compliance'] # Seriesに変換

    # 予測したclasses_を確認
    print(model.classes_)
    # 返却するSeriesが61001件であることを確認
    print(answer.shape)

    return answer# Your answer here

In [63]:
ret = blight_model()
# print(ret)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


                       address  violation_zip_code violation_street_name  \
0       2900 tyler, Detroit MI                 NaN                 TYLER   
1     4311 central, Detroit MI                 NaN               CENTRAL   
2  1449 longfellow, Detroit MI                 NaN            LONGFELLOW   
3  1441 longfellow, Detroit MI                 NaN            LONGFELLOW   
4   2449 churchill, Detroit MI                 NaN             CHURCHILL   

  mailing_address_str_name     city state zip_code  
0                S. WICKER  CHICAGO    IL    60606  
1       Martin Luther King  Detroit    MI    48208  
2                 P.O. BOX  DETROIT    MI    48223  
3                ST. CLAIR  DETROIT    MI    48214  
4                CHURCHILL  DETROIT    MI    48206  
lat                float64
lon                float64
agency_name         object
state               object
country             object
disposition         object
fine_amount        float64
discount_amount    float64
clean_up_